# Iterative Solvers with Linear Operators

Here we demo how to solve data-sparse linear operators with an iterative solver.  Essentially we'd like to solve
$$Ax = b$$
for $x$, where $A$ may have some special structure

The two packages we'll use are
* [LinearOperators.jl](https://juliasmoothoptimizers.github.io/LinearOperators.jl/latest/index.html)
* [IterativeSolvers.jl](https://juliamath.github.io/IterativeSolvers.jl/latest/)

In the exercises, you can also try out
* [RandomizedLinAlg.jl](https://haampie.github.io/RandomizedLinAlg.jl/latest/)

Which allows you to use randomized algorithms on linear operators.  Note that this package is still not very well developed.

In [2]:
using Pkg; Pkg.add(["IterativeSolvers","LinearOperators"])
using LinearOperators, IterativeSolvers, LinearAlgebra

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 % Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]


LinearOperators let you work with low-rank and otherwise sparse objects as if they were matrices, but with basically no performance penalty for doing so.

In [3]:
n = 1000
v = randn(n,1)
v /= norm(v)
opV = LinearOperator(v)
A = v*v'
opA = opV * opV'
;

In [5]:
x = randn(n)
@time b1 = A * x
@time b2 = opA*x 
@time b3 = dot(v,x)*v
@show norm(b1-b2)
@show norm(b2-b3)
;

  0.002600 seconds (5 allocations: 8.094 KiB)
  0.000019 seconds (6 allocations: 8.188 KiB)
  0.000017 seconds (6 allocations: 8.109 KiB)
norm(b1 - b2) = 4.2090860693657047e-16
norm(b2 - b3) = 0.0


In [6]:
#comparing how LinearOperators do when compared to the equivalent vectorized code
function f1(n)
    local x = randn(n)
    for i=1:10*n
        local v = randn(n,1)
        local v /= norm(v)
        local opV = LinearOperator(v)
        local b2 = opA*x
    end
end

function f2(n)
    local x = randn(n)
    for i=1:10*n
        local v = randn(n)
        local v /= norm(v)
        local b3 = dot(v,x)*v
    end
end

f2 (generic function with 1 method)

In [8]:
@time f1(1000)
@time f2(1000)

  0.258557 seconds (40.01 k allocations: 233.467 MiB, 12.30% gc time)
  0.273919 seconds (30.00 k allocations: 232.552 MiB, 14.11% gc time)


## Use with IterativeSolvers package

In [9]:
using Random
Random.seed!(1)
@show c = sqrt(2*log(n))
opA2 = (2*c)*opA + Diagonal(abs.(randn(n))) # spiked model
opA2.symmetric=true
opA2.hermitian=true
opA2

c = sqrt(2 * log(n)) = 3.7169221888498383


Linear operator
  nrow: 1000
  ncol: 1000
  eltype: Float64
  symmetric: true
  hermitian: true



In [18]:
x_true = randn(n)
b = opA2 * x_true
@time x_est_minres = minres(opA2, b)
@time x_est_cg = cg(opA2, b)
@show norm(b - opA2*x_est_minres)
@show norm(b - opA2*x_est_cg)
;

  0.013801 seconds (1.24 k allocations: 7.643 MiB, 80.02% gc time)
  0.002155 seconds (1.29 k allocations: 7.970 MiB)
norm(b - opA2 * x_est_minres) = 4.036826124819918e-7
norm(b - opA2 * x_est_cg) = 3.754567418356935e-7


As a comparison, here's how solving the dense matrix with backslash goes:

In [15]:
Random.seed!(1)
A2 = (2*c)*A + Diagonal(abs.(randn(n)))
@time x_est = A2\b
@show norm(b - A2*x_est)
;

  0.126188 seconds (9 allocations: 7.645 MiB, 65.32% gc time)
norm(b - A2 * x_est) = 2.019170560589031e-14


## Exercises/Extras

If you're interested, try out one or more of the following exercises:

1. Try out the randomized linear algebra package [RandomizedLinAlg.jl](https://haampie.github.io/RandomizedLinAlg.jl/latest/).  Try using the ```rnorms``` function to estimate the matrix norm.

2. Try out the Krylov methods package [Krylov.jl](https://github.com/JuliaSmoothOptimizers/Krylov.jl), or mess with matrix exponentials with [Expokit.jl](https://github.com/acroy/Expokit.jl).

3. Make a plot of how long it takes to solve $Ax = b$ for $A$ diagonal + rank-1, for various sizes of problems. Estimate how long it would take to solve the equivalent problem using the full matrix

4. You can also use sparse matrices as LinearOperators, and with iterative solvers.  Use [sprand](https://docs.julialang.org/en/stable/stdlib/arrays/#Base.SparseArrays.sprand) to generate sparse matrices of various sizes and try using [gmres](https://juliamath.github.io/IterativeSolvers.jl/latest/linear_systems/gmres.html) to solve some linear systems.

5. Check out the [tutorial](https://juliasmoothoptimizers.github.io/LinearOperators.jl/latest/tutorial.html#Using-functions-1) on how to use functions as linear operators